## Ejecución del pipeline OULAD (versión robusta y eficiente)

Puedes ejecutar el pipeline completo desde terminal o desde una celda de Jupyter/Colab:

```python
!python ../src/oulad_pipeline.py --max_rows 20000
```

- El parámetro `--max_rows` permite trabajar con un subconjunto aleatorio del dataset para pruebas rápidas.
- Todos los resultados se guardan en la carpeta `results/`.


# Ejecución avanzada del pipeline OULAD

Este notebook ejecuta el pipeline completo: carga de datos, preprocesamiento, EDA, modelado supervisado/no supervisado, búsqueda de hiperparámetros, reducción de dimensionalidad e interpretación de resultados.

In [ ]:
# Importar librerías necesarias y módulos del pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, mean_squared_error, r2_score
from xgboost import XGBClassifier
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
import os
import sys
sys.path.append('../src')
from src.oulad_pipeline import OULADDBConnector, OULADPreprocessor, OULADEDA, OULADModel, OULADInterpreter
from src.db_queries import read_table


## Cargar y preparar los datos OULAD

En esta sección se cargan los datos desde la base de datos MySQL (o archivos CSV) y se realiza el join necesario para obtener la variable objetivo.

In [ ]:
# Cargar datos desde la base de datos MySQL y realizar join
output_dir = '../results'
db = OULADDBConnector()
db.test_connection()
df_vle = read_table('studentVle', db)
df_info = read_table('studentInfo', db)
df_merged = pd.merge(df_vle, df_info[['id_student', 'code_presentation', 'final_result']],
                    on=['id_student', 'code_presentation'], how='inner')
print(f'Registros tras el join: {len(df_merged):,}')
df_merged.head()

## Limpieza y preprocesamiento de datos

Se aplica la función `clean` de OULADPreprocessor para tratar nulos, duplicados y outliers.

In [ ]:
# Limpieza y preprocesamiento de datos
preprocessor = OULADPreprocessor()
df_clean, num_vars = preprocessor.clean(df_merged)
print('Variables numéricas:', num_vars)
df_clean.head()

## Ingeniería de características

Transformación y codificación de variables usando `feature_engineering` de OULADPreprocessor.

In [ ]:
# Ingeniería de características
# rq=1 para clasificación binaria, ajusta según tu caso
rq = 1
df_feat = preprocessor.feature_engineering(df_clean, rq=rq)
df_feat.head()

## Análisis Exploratorio de Datos (EDA)

En esta sección se realiza un análisis exploratorio utilizando la clase `OULADEDA`. Se incluyen análisis univariados, bivariados, boxplots, correlación y kurtosis para comprender la distribución y relaciones de las variables.

In [ ]:
# EDA: Análisis univariado, bivariado, boxplots, correlación y kurtosis
from src.oulad_pipeline import OULADEDA

df = preprocessor.df  # Usar el DataFrame preprocesado
eda = OULADEDA(df)

# Análisis univariado de variables numéricas
eda.univariate_analysis(columns=['age_band', 'num_of_prev_attempts', 'studied_credits', 'final_result'])

# Análisis bivariado entre variables relevantes
eda.bivariate_analysis(x='age_band', y='final_result')

# Boxplots para variables numéricas
eda.boxplot(columns=['studied_credits', 'num_of_prev_attempts'])

# Matriz de correlación
eda.correlation_matrix()

# Kurtosis de variables numéricas
eda.kurtosis_analysis(columns=['studied_credits', 'num_of_prev_attempts'])

## Modelado Supervisado: Clasificación y Regresión

En esta sección se ejemplifica el uso de la clase `OULADModel` para tareas de clasificación y regresión, incluyendo la exportación de métricas y visualización de resultados.

In [ ]:
# Modelado supervisado: Clasificación y regresión
modeler = OULADModel()
# Definir variables predictoras y objetivo para clasificación
y_class = df_feat['final_result']
X_class = df_feat.drop(columns=['final_result'])
# División de datos
X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42, stratify=y_class)
# Entrenamiento y predicción con RandomForest
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
pipe = Pipeline([
    ('smote', SMOTE()),
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(n_jobs=-1, random_state=42))
])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
# Interpretación y exportación de métricas
interpreter = OULADInterpreter()
cm, metrics = interpreter.export_metrics(y_test, y_pred, output_dir)
interpreter.plot_confusion_matrix(y_test, y_pred, output_dir)
interpreter.plot_feature_importances(pipe.named_steps['classifier'], X_class.columns, output_dir)
print('Métricas y visualizaciones exportadas a la carpeta results/')

## Modelado No Supervisado: Clustering y Visualización

En esta sección se utiliza la funcionalidad de clustering del pipeline para agrupar estudiantes y visualizar los clusters resultantes.

In [ ]:
# Clustering y visualización de clusters
# (Ejemplo con KMeans, se puede ajustar el número de clusters)
from src.oulad_pipeline import OULADModel

clustering = OULADModel(model_type='clustering', model_name='KMeans', n_clusters=3)
clustering.train(X_class)  # Usar las mismas features que en clasificación
labels = clustering.predict(X_class)
clustering.plot_clusters(X_class, labels)
clustering.export_cluster_assignments('results/cluster_assignments.csv')

## Búsqueda de Hiperparámetros y Reducción de Dimensionalidad

En esta sección se ejemplifica la búsqueda de hiperparámetros (grid search) para RandomForest/XGBoost y la reducción de dimensionalidad mediante proyección aleatoria, incluyendo visualización e interpretación de resultados.

In [ ]:
# Grid search de hiperparámetros y reducción de dimensionalidad
from src.oulad_pipeline import OULADModel

# Grid search para RandomForest
param_grid_rf = {'n_estimators': [50, 100], 'max_depth': [5, 10]}
clf_rf = OULADModel(model_type='classification', model_name='RandomForest')
gs_results_rf = clf_rf.grid_search(X_class, y_class, param_grid=param_grid_rf)
print('Mejores parámetros RF:', gs_results_rf['best_params'])

# Grid search para XGBoost
param_grid_xgb = {'n_estimators': [50, 100], 'max_depth': [3, 6]}
clf_xgb = OULADModel(model_type='classification', model_name='XGBoost')
gs_results_xgb = clf_xgb.grid_search(X_class, y_class, param_grid=param_grid_xgb)
print('Mejores parámetros XGBoost:', gs_results_xgb['best_params'])

# Reducción de dimensionalidad (proyección aleatoria)
clf_rf.reduce_dimensionality(X_class, method='random_projection', n_components=2)
clf_rf.plot_reduced_space(X_class, y_class)

## Interpretación y Visualización de Resultados

En esta sección se muestran ejemplos de interpretación de resultados, visualización de métricas, exportación de predicciones y análisis de importancia de variables.

In [ ]:
# Interpretación y visualización de resultados adicionales
# Exportar y_test, y_pred y métricas manuales ya realizado arriba
# Mostrar matriz de confusión y métricas
import pandas as pd
display(pd.read_csv(f'{output_dir}/metrics_manual.csv'))
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(f'{output_dir}/confusion_matrix.png')
plt.imshow(img)
plt.axis('off')
plt.show()
# Importancia de variables
img2 = mpimg.imread(f'{output_dir}/feature_importances.png')
plt.imshow(img2)
plt.axis('off')
plt.show()

## Validación y anexos para artículo científico

A continuación se muestran ejemplos de cómo validar outputs y preparar anexos para la documentación científica, incluyendo la exportación de predicciones, métricas y visualizaciones clave generadas por el pipeline.

In [ ]:
# Validación de outputs y generación de anexos para el artículo científico

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Cargar y mostrar las predicciones y métricas exportadas
y_pred_df = pd.read_csv(f'{output_dir}/y_test_y_pred.csv')
metrics_manual = pd.read_csv(f'{output_dir}/metrics_manual.csv')
print("Primeras filas de y_test vs y_pred:")
display(y_pred_df.head())
print("Métricas manuales calculadas:")
display(metrics_manual)

# Visualizar matriz de confusión y guardar como anexo
img_cm = mpimg.imread(f'{output_dir}/confusion_matrix.png')
plt.figure(figsize=(5,4))
plt.imshow(img_cm)
plt.axis('off')
plt.title('Matriz de Confusión (Anexo)')
plt.show()

# Visualizar importancia de variables y guardar como anexo
img_fi = mpimg.imread(f'{output_dir}/feature_importances.png')
plt.figure(figsize=(10,6))
plt.imshow(img_fi)
plt.axis('off')
plt.title('Importancia de Variables (Anexo)')
plt.show()

# Nota: Puedes adjuntar los archivos CSV y PNG generados en la carpeta results/ como anexos en tu artículo.

## Checklist de reproducibilidad y validación final

- [ ] El entorno se puede instalar con `pip install -r requirements.txt` sin errores.
- [ ] La configuración de conexión a MySQL está documentada y es clara (`config/settings.py`).
- [ ] El pipeline se ejecuta correctamente desde terminal con `python src/oulad_pipeline.py`.
- [ ] Todos los archivos de resultados (`metrics_manual.csv`, `y_test_y_pred.csv`, `confusion_matrix.png`, etc.) se generan en la carpeta `results/`.
- [ ] Los archivos exportados son legibles y adecuados para anexos científicos.
- [ ] El código está modularizado y documentado para facilitar su uso y extensión.
- [ ] El flujo OSEMN está cubierto de inicio a fin.

> Marca cada punto al validar en un entorno limpio o al preparar la entrega/académica.

## Visualizaciones avanzadas: Curva ROC, SHAP y análisis de errores

En esta sección se muestran ejemplos de visualización avanzada para interpretación y análisis de modelos:
- Curva ROC para clasificación binaria
- Interpretabilidad con SHAP (modelos de árbol)
- Análisis de errores (matriz de confusión normalizada)


In [ ]:
# Curva ROC para clasificación binaria
from sklearn.metrics import roc_curve, auc, RocCurveDisplay
import matplotlib.pyplot as plt

if 'y_test' in locals() and 'y_pred' in locals():
    if hasattr(pipe.named_steps['classifier'], "predict_proba"):
        y_score = pipe.named_steps['classifier'].predict_proba(X_test)[:,1]
    else:
        y_score = pipe.decision_function(X_test)
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curva ROC')
    plt.legend(loc="lower right")
    plt.show()

# Interpretabilidad con SHAP (solo para modelos de árbol)
try:
    import shap
    explainer = shap.TreeExplainer(pipe.named_steps['classifier'])
    shap_values = explainer.shap_values(X_test)
    shap.summary_plot(shap_values, X_test, plot_type="bar")
    shap.summary_plot(shap_values, X_test)
except Exception as e:
    print("SHAP no disponible o el modelo no es compatible:", e)

# Matriz de confusión normalizada
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_estimator(pipe, X_test, y_test, normalize='true', cmap='Blues')
plt.title('Matriz de confusión normalizada')
plt.show()


In [ ]:
# Visualización y ranking de importancia de variables
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Mostrar gráfico si existe
feature_importances_path = '../results/feature_importances.png'
if os.path.exists(feature_importances_path):
    img = mpimg.imread(feature_importances_path)
    plt.figure(figsize=(10,6))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Importancia de variables (RandomForest)')
    plt.show()
else:
    print('No se encontró la imagen feature_importances.png. Mostrando ranking textual:')
    if hasattr(pipe.named_steps['classifier'], 'feature_importances_'):
        importances = pipe.named_steps['classifier'].feature_importances_
        indices = importances.argsort()[::-1]
        for i in indices:
            print(f"{X_class.columns[i]}: {importances[i]:.4f}")
    else:
        print('El modelo no tiene el atributo feature_importances_. Usa un modelo de árbol para obtener importancias.')
